# ***Distribución BAÑOS***

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "alfa" in nombre and "tulancingo" in nombre

    #if nombre.startswith("df_") and "tulancingo" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_abril_2024_tulancingo',
 'df_alfa_agosto_2024_tulancingo',
 'df_alfa_febrero_2024_tulancingo',
 'df_alfa_julio_2024_tulancingo',
 'df_alfa_junio_2024_tulancingo',
 'df_alfa_marzo_2024_tulancingo',
 'df_alfa_mayo_2024_tulancingo',
 'df_alfa_q_jul_2022_tulancingo',
 'df_alfa_q_jul_2023_tulancingo',
 'df_alfa_q_jun_2023_tulancingo',
 'df_alfa_q_mar_2023_tulancingo',
 'df_alfa_q_may_2022_tulancingo',
 'df_alfa_q_may_2023_tulancingo',
 'df_alfa_q_oct_2022_tulancingo',
 'df_alfa_q_oct_2023_tulancingo',
 'df_alfa_q_sep_2023_tulancingo']

In [2]:
df_alfa_agosto_2024_tulancingo.columns

Index(['id', 'Categoria', 'Precio', 'propiedad', 'metros_total',
       'metros_construido', 'precio_m2_terreno', 'precio_m2_construido',
       'tiempo_de_publicacion', 'Estado', 'Tipo', 'Estacionamientos',
       'Recamaras', 'Banos', 'Medio_banos', 'Banos_Total', 'Seguridad',
       'Colonia', 'CP', 'url'],
      dtype='object')

In [3]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'Precio':'precio','Categoria':'categoria','Banos_Total':'baño_total','id_jul_23':'id','m2_contruido': 'm2_construido','m_construido': 'm2_construido','Metros_construido':'m2_construido','segmento':'categoria','Category':'categoria','m_total':'m2_total','m_construido':'m2_construido',}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")

Columnas renombradas en los DataFrames filtrados.


In [4]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id','categoria','baño_total','precio']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

## Total de baños

In [5]:
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500000, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),  
    "B2": (2750001, 3000000),  
    "B3": (3000001, 3250000),  
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Función para asignar la categoría según el precio
def asignar_categoria(precio):
    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
        if limite_inferior <= precio <= limite_superior:
            return categoria
    return None  # En caso de que el precio no caiga en ningún rango (caso raro)

In [6]:
import numpy as np
for df in dataframes_list:    
    condition = (df['baño_total'] % 1 == 0.5) # Valores que contienen .5    
    # Convertir al número entero anterior
    df.loc[condition, 'baño_total'] = np.floor(df['baño_total'])    
    print(df['baño_total'].unique())
    df = df.drop(columns=['categoria'])
    df['categoria'] = df['precio'].apply(asignar_categoria)

[5 0 1 4 2 3]
[4. 3. 1. 2.]
[1 2 3 4 5 0]
[2. 3. 1. 4. 5. 6.]
[1 2 3 4 5 7]
[1 2 3 4 5 7]
[1 2 3 4]
[1 2 3 4 5]
[1 2 3 0 4 5]
[0 1 2 3 4 5]
[2 1 3 4 5]
[1 2 3 5 4 6 7]
[    1     3     2 45048 45047     4 45049]
[1 2 3 4 5 6]
[2 3 1 4 5]
[2 3 1 4 5]


In [7]:
def procesar_dataframes(dataframes_list, nombres_df_filtrados):
    for i, df in enumerate(dataframes_list):
        # Crear una nueva columna con la primera letra de la categoría
        df['segmento'] = df['categoria'].str[0]
        df.drop(columns=['categoria'], inplace=True)
        df['baño_total'] = df['baño_total'].clip(lower=1,upper=5) # Limitar los valores de 'baño_total' a un máximo de 8
        #df['baño_total'] = pd.cut(df['baño_total'], bins=bins, labels=labels, right=False)

        # Agrupar y contar los datos
        tabla = df.groupby(['segmento', 'baño_total']).size().unstack(fill_value=0)

        # Imprimir el nombre del DataFrame y la tabla resultante
        print(f"Nombre del DataFrame: {nombres_df_filtrados[i]}")
        print(tabla)

# Procesar los DataFrames de la lista
procesar_dataframes(dataframes_list, nombres_df_filtrados)

C:\Users\yoe11\AppData\Local\Temp\ipykernel_16296\1729679471.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['segmento'] = df['categoria'].str[0]
C:\Users\yoe11\AppData\Local\Temp\ipykernel_16296\1729679471.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['categoria'], inplace=True)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_16296\1729679471.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

Nombre del DataFrame: df_alfa_abril_2024_tulancingo
baño_total   1   2  3
segmento             
A            4   0  0
B            6   3  2
C            7   2  0
D           13   8  2
E           48  20  4
S            3   0  0
Nombre del DataFrame: df_alfa_agosto_2024_tulancingo
baño_total  1.0  2.0  3.0  4.0
segmento                      
A             0    3    2    0
B             1    5    2    0
C             2    5    1    1
D             8    8    2    0
E            28   35   10    0
S             0    2    2    0
Nombre del DataFrame: df_alfa_febrero_2024_tulancingo
baño_total   1   2  3  4  5
segmento                   
A            2   3  0  0  3
B            3   3  3  0  0
C            5   2  2  0  0
D           11   5  1  0  0
E           49  29  4  1  0
S            4   0  3  0  0
Nombre del DataFrame: df_alfa_julio_2024_tulancingo
baño_total  1.0  2.0  3.0  4.0  5.0
segmento                           
A             1    2    3    0    0
B             1    5    2    0   

In [8]:
# Crear una nueva función para sumar las tablas de todos los DataFrames
def sumar_tablas(dataframes_list):
    # Inicializar una tabla vacía para almacenar la suma
    tabla_suma = None
    
    # Iterar sobre cada DataFrame en la lista
    for df in dataframes_list:
        # Agrupar y contar los datos en el DataFrame actual
        tabla_actual = df.groupby(['segmento', 'baño_total']).size().unstack(fill_value=0)
        
        # Sumar la tabla actual al acumulador (tabla_suma)
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    # Imprimir la tabla resultante de la suma
    print("Tabla general de frecuencia:")
    print(tabla_suma)

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
sumar_tablas(dataframes_list)

Tabla general de frecuencia:
baño_total    1.0    2.0   3.0   4.0   5.0
segmento                                  
A            24.0   20.0  27.0  16.0  13.0
B            33.0   43.0  53.0   8.0   0.0
C            49.0  128.0  54.0   7.0   1.0
D           195.0  229.0  90.0   2.0   2.0
E           395.0  244.0  47.0   3.0   0.0
L             0.0    0.0   0.0   2.0   2.0
S            20.0   30.0  53.0  22.0  10.0


In [9]:
import os
import plotly.graph_objects as go
import plotly.io as pio

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#F79CB9']
orden_segmentos=["E", "D", "C", "B","A", "S", ]

def sumar_tablas(dataframes_list):
    tabla_suma = None
    for df in dataframes_list:
        # Aplica el orden a la columna 'segmento'
        df['segmento'] = pd.Categorical(df['segmento'], categories=orden_segmentos, ordered=True)
        
        tabla_actual = df.groupby(['segmento', 'baño_total']).size().unstack(fill_value=0)
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    print("Tabla general de frecuencia:")
    print(tabla_suma)
    
    fig = graficar_tabla(tabla_suma)
    guardar_grafico_como_html(fig, 'g_bar_banos', carpeta='assets/graficas') # Para guardar la gráfica como archivo HTML
    return fig

def graficar_tabla(tabla):
    segmentos = tabla.index
    banos = tabla.columns
    
    fig = go.Figure()
    
    for i, segmento in enumerate(segmentos):
        color = colores[i % len(colores)]
        fig.add_trace(go.Bar(
            x=banos,
            y=tabla.loc[segmento],
            name=segmento,
            marker_color=color
        ))
    
    fig.update_layout(
        xaxis_title="Número de baños",        
        yaxis=dict(
            title='Número de casas',
            gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
            gridwidth=1,
        ),
        legend_title="Segmento",
        margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
        barmode='group',
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
        ),
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    )    
    
    fig.show()    
    return fig

# Guardar la gráfica como archivo HTML en la carpeta especificada
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Guardar la gráfica como archivo HTML
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
fig = sumar_tablas(dataframes_list)

C:\Users\yoe11\AppData\Local\Temp\ipykernel_16296\139836840.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['segmento'] = pd.Categorical(df['segmento'], categories=orden_segmentos, ordered=True)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_16296\139836840.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tabla_actual = df.groupby(['segmento', 'baño_total']).size().unstack(fill_value=0)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_16296\139836840.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

Tabla general de frecuencia:
baño_total  1.0  2.0  3.0  4.0   5.0
segmento                            
E           395  244   47    3   0.0
D           195  229   90    2   2.0
C            49  128   54    7   1.0
B            33   43   53    8   0.0
A            24   20   27   16  13.0
S            20   30   53   22  10.0


In [10]:
def sumar_tablas(dataframes_list):
    tabla_suma = None
    
    for df in dataframes_list:
        tabla_actual = df.groupby('baño_total').size()
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    return tabla_suma

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
tabla_suma = sumar_tablas(dataframes_list)
print("Tabla general total de registros:")
print(tabla_suma)


Tabla general total de registros:
baño_total
1.0    734.0
2.0    714.0
3.0    337.0
4.0     70.0
5.0     33.0
dtype: float64


In [11]:
# Calcular el total de registros
total_registros = tabla_suma.sum()
# Calcular el porcentaje que representa cada número de baños único del total
porcentajes = (tabla_suma / total_registros) * 100

# Imprimir los porcentajes
print("Porcentaje de cada número de baños único en relación con el total de registros:", total_registros)
print(porcentajes)


Porcentaje de cada número de baños único en relación con el total de registros: 1888.0
baño_total
1.0    38.877119
2.0    37.817797
3.0    17.849576
4.0     3.707627
5.0     1.747881
dtype: float64


In [13]:
import os
import plotly.graph_objects as go
import plotly.io as pio

# Lista de colores
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#F79CB9','#E50CB6','#4225CF']

# Función para sumar tablas
def sumar_tablas(dataframes_list):
    tabla_suma = None
    
    for df in dataframes_list:
        tabla_actual = df.groupby('baño_total').size()
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    return tabla_suma

# Función para calcular porcentajes
def calcular_porcentajes(tabla):
    # Calcular los porcentajes
    tabla_porcentajes = tabla / tabla.sum() * 100
    return tabla_porcentajes

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
tabla_suma = sumar_tablas(dataframes_list)
porcentajes = calcular_porcentajes(tabla_suma) # Calcular porcentajes

# Crear el gráfico de pastel con colores especificados
figURE = go.Figure(data=[go.Pie(labels=porcentajes.index, values=porcentajes, sort=False, marker=dict(colors=colores))])
figURE.update_layout( title_x=0.5, title_font_size=20,margin=dict(l=10, r=10, t=10, b=10),  )
figURE.update_traces(rotation=45) # Ángulo de inicio de las porciones del gráfico

# Exportar gráfica como archivo HTML y mostrar la figura
def guardar_grafico_como_html(figURE, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
   # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Guardar la gráfica como archivo HTML
    pio.write_html(figURE, f'{carpeta}/{nombre_archivo}.html', config=config)

# Exportar
guardar_grafico_como_html(figURE, 'g_pie_banos', carpeta='assets/graficas')

# Mostrar la figura
figURE.show()